In [6]:
import os
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline


import glob
from astropy.io import fits
from astropy.stats import sigma_clip
from scipy.ndimage import interpolation as interp

#from skimage.feature.register_translation import (register_translation, _upsampled_dft)

# import aotools
import astroscrappy
from scipy.optimize import leastsq
# import ccdproc
import math

In [7]:
from scipy.ndimage.filters import median_filter, gaussian_filter


def guess_gaussian_parameters(d):
    """
    This image guesses the maximum intensity of the 
    image by obtaining a smoothed version of the image 
    via median + gaussian filtering. Then, finds the 
    maximum of the smoothed image. Using the median-filtered 
    image, the algorithm also estimates the width of the PSF and 
    with this value and an estimate of the volume, the amplitude of 
    such gaussian.

    Input

    d       Numpy array that contains the values of the pixels.

    Output

    x0      x coordinate of the image maximum
    
    y0      y coordinate of the image maximum

    sigma   Width of the PSF

    A       Estimated amplitude of the gaussian function
    """

    # First, smooth the image with a median filter. For this, find 
    # the optimal window as the square-root of the geometric mean 
    # between the sizes of the array. This step is good to kill outliers:
    window = int(np.sqrt(np.sqrt(d.shape[0] * d.shape[1])))
    if window % 2 == 0:
        window = window + 1

        # Originally there was a median filter here, but it really adds a lot of time to the routine
    #d_mfiltered = median_filter(d,size = window)
    # Going to try without it to see if it makes a significant difference for vetoing
    d_gfiltered = gaussian_filter(d, sigma=window)

    # Now, find the maximum of this image:
    y0, x0 = np.where(d_gfiltered == np.max(d_gfiltered))

    # Take the first element. This helps in two cases: (1) only one maximum has 
    # been found, the outputs are numpy arrays and you want to extract the numbers 
    # and (2) in case there are several maximums (this has not happened so 
    # far but could in principle), pick the first of them:
    y0 = y0[0]
    x0 = x0[0]

    # Now estimate the width of the PSF by taking a "cross-plot" using the 
    # maximum values found:
    x_cut = d[:, int(x0)]
    sigma_x = (np.sum(x_cut * (np.abs(np.arange(len(x_cut)) - y0))) / np.sum(x_cut)) / 3.
    y_cut = d[int(y0), :]
    sigma_y = (np.sum(y_cut * (np.abs(np.arange(len(y_cut)) - x0))) / np.sum(y_cut)) / 3.
    sigma = np.sqrt(sigma_x * sigma_y)

    # (Under) estimate amplitude assuming a gaussian function:
    A = (np.sum(d - np.median(d)) / (2. * np.pi * sigma ** 2))

    return x0, y0, sigma, 2. * A


/tmp/ipykernel_16855/3376877513.py:1: DeprecationWarning: Please use `median_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter, gaussian_filter
/tmp/ipykernel_16855/3376877513.py:1: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import median_filter, gaussian_filter


In [8]:
datadir = '/home/tehan/PycharmProjects/Shane-AO-Reduction/Raw_Data/data-2023-07-03-AO-Paul.Robertson/'
os.chdir(datadir)

#These are the id numbers of the fitz files
firstID = 1;
lastID = 9999;

##Let's go through our images and remove the bad frames

all_object_list = []

veto_list = []

#Let's just make a list of every object
for ID in range(firstID, lastID + 1):
    try:
        if ID < 100:
            if ('s00' + str(ID) + '.fits'):
                header = fits.getheader('s00' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s00' + str(ID) + '.fits')


        elif 99 < ID < 1000:
            if ('s0' + str(ID) + '.fits'):
                header = fits.getheader('s0' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s0' + str(ID) + '.fits')

        elif 999 < ID < 10000:
            if ('s' + str(ID) + '.fits'):
                header = fits.getheader('s' + str(ID) + '.fits')
                obj = header['OBJECT']
                all_object_list.append('s' + str(ID) + '.fits')

    except OSError:
        continue
    except KeyError:
        continue

flat_obj_list = []
#let's remove the darks
for object in all_object_list:
    header = fits.getheader(object)
    obj = header['OBJECT']
    if obj == 'dark':
        continue
    else:
        flat_obj_list.append(object)




In [9]:
#Now we go through each object here, and we make a list of all the bad ones

veto_list = []

for obj in flat_obj_list:
    header = fits.getheader(obj)
    try:
        image_data = fits.getdata(obj)
        center=(1100,730)
        image_window = image_data[(center[1] - 200):(center[1] + 200), (center[0] - 200):(center[0] + 200)]  # we use the window we care about
        # shrink the window to avoid companion

        print(obj)
        print(header['OBJECT'])
        if 'flat' in header['OBJECT'].lower():
            cent_counts = np.mean(image_window)
            print('Flat!')
        else:

            x0, y0, sigma, A = guess_gaussian_parameters(
                image_window)  #need to do this for the window, not the whole image

            print(image_window[y0,x0])
            #now that we have the centroid, let's make a window about the centroid, and take the average
            centroid_window = image_window[y0 - 1:y0 + 1, x0 - 1:x0 + 1]
            cent_counts = np.mean(centroid_window)  #let's try max instead of mean

        print(cent_counts)

        if cent_counts > 25000:  #Setting our max counts at 25000 here
            veto_list.append(obj)
            print('Appended!')
        elif cent_counts < 500:  #Setting our minimum counts at 6500 here
            veto_list.append(obj)
            print('Appended!')
        elif math.isnan(cent_counts) == True:
            veto_list.append(obj)
            print('Appended!')
        else:
            continue
    except ValueError:
        continue


#Just median filter: ~94 s, appears accurate
#Just Gaussian filter: ~ 1 s, appears accurate



s0063.fits
Dome Flat
Flat!
8707.744
s0064.fits
Dome Flat
Flat!
8739.498
s0065.fits
Dome Flat
Flat!
8753.716
s0066.fits
Dome Flat
Flat!
8751.45
s0067.fits
Dome Flat
Flat!
8761.604
s0068.fits
Dome Flat
Flat!
8755.997
s0069.fits
Dome Flat
Flat!
8760.683
s0070.fits
Dome Flat
Flat!
10750.927
s0071.fits
Dome Flat
Flat!
10757.615
s0072.fits
Dome Flat
Flat!
10753.35
s0073.fits
Dome Flat
Flat!
10767.606
s0074.fits
Dome Flat
Flat!
10763.053
s0075.fits
Dome Flat
Flat!
10761.253
s0076.fits
Dome Flat
Flat!
10768.632
s0077.fits
Dome Flat
Flat!
9807.472
s0078.fits
Dome Flat
Flat!
9803.755
s0079.fits
Dome Flat
Flat!
9797.306
s0080.fits
Dome Flat
Flat!
9810.305
s0081.fits
Dome Flat
Flat!
9802.606
s0082.fits
Dome Flat
Flat!
9804.295
s0083.fits
Dome Flat
Flat!
9793.827
s0084.fits
Dome Flat
Flat!
7275.8022
s0085.fits
Dome Flat
Flat!
7259.971
s0086.fits
Dome Flat
Flat!
7263.2974
s0087.fits
Dome Flat
Flat!
7256.0903
s0088.fits
Dome Flat
Flat!
7267.541
s0089.fits
Dome Flat
Flat!
7266.4663
s0090.fits
Dome Fla

/tmp/ipykernel_16855/3376877513.py:54: RuntimeWarning: invalid value encountered in double_scalars
  sigma_x = (np.sum(x_cut * (np.abs(np.arange(len(x_cut)) - y0))) / np.sum(x_cut)) / 3.
/tmp/ipykernel_16855/3376877513.py:56: RuntimeWarning: invalid value encountered in double_scalars
  sigma_y = (np.sum(y_cut * (np.abs(np.arange(len(y_cut)) - x0))) / np.sum(y_cut)) / 3.


7753.8438
7678.6875
s0312.fits
TOI-6034
0.0
nan
Appended!
s0313.fits
TOI-6034
0.0
nan
Appended!
s0314.fits
TOI-6034
7200.3125
7148.922
s0315.fits
TOI-6034
6318.9062
nan
Appended!
s0316.fits
TOI-6034
0.0
nan
Appended!
s0317.fits
TOI-6022
6417.1875
nan
Appended!
s0318.fits
TOI-6022
8548.8125
nan
Appended!
s0319.fits
TOI-6022
8482.8125
nan
Appended!
s0320.fits
TOI-6022
8517.375
nan
Appended!
s0321.fits
TOI-6022
8463.719
nan
Appended!
s0322.fits
TOI-6022
9714.0625
9694.75
s0323.fits
TOI-6022
9295.25
9380.984
s0324.fits
TOI-6022
8513.75
nan
Appended!
s0325.fits
TOI-6022
10516.219
10446.25
s0326.fits
TOI-6022
9640.625
9700.391
s0327.fits
TIC_155113423
26347.188
26482.984
Appended!
s0328.fits
TIC_155113423
13867.625
13899.0625
s0329.fits
TIC_155113423
9805.281
9886.906
s0330.fits
TIC_155113423
9085.375
9190.094
s0331.fits
TIC_155113423
10242.75
10062.859
s0332.fits
TIC_155113423
10855.5625
10900.094
s0333.fits
TIC_155113423
9425.8125
9071.391
s0334.fits
TIC_155113423
10387.8125
10389.031
s033

In [10]:
#Write it to a text file for later
with open('veto_list.txt', 'w') as f:
    for item in veto_list:
        f.write("%s\n" % item)